In [437]:
import pandas as pd
import numpy as np
import re

In [438]:
dataset = pd.io.parsers.read_csv('a1_d3.txt', sep='\t',names=['data','value'])
zeroes = (dataset['value']==0).sum(axis=0)
ones = (dataset['value']==1).sum(axis=0)

# Preprocessing

In [439]:
stop = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
dataset['data'] = dataset['data'].str.lower()
dataset['data'] = dataset['data'].str.replace(r'[-,\.&!\+\';0-9"\(\)#@$%^&?~`:'']+','',regex=True)
dataset['data'] = list(map(str.split, dataset['data']))
dataset['data'] = dataset['data'].apply(lambda x: [item for item in x if item not in stop])
# values = dataset.data.str.split(expand=True).stack()#value_counts()

In [440]:
dataset = dataset.sample(frac=1)
train_set = dataset[:int(0.8*len(dataset))]
test_set = dataset[int(0.8*len(dataset)):]
x_train = train_set['data']
y_train = train_set['value']
x_test = test_set['data']
y_test = test_set['value']
zeroes = (y_train==0).sum(axis=0)
ones = (y_train==1).sum(axis=0)
print(zeroes,ones)

406 394


In [441]:
x_train1 = x_train[y_train==1]
x_train0 = x_train[y_train==0]

In [442]:
x_train1

586                             [battery, holding, well]
518                          [phone, sturdy, waterproof]
125                                       [great, audio]
252                 [attractive, appears, good, quality]
208                              [good, transmit, audio]
                             ...                        
185                                         [incredible]
389    [company, shipped, product, promptly, works, w...
758                    [sound, quality, excellent, well]
423         [im, using, iriver, spinn, case, fits, fine]
899    [pairing, two, devices, easy, barely, took, co...
Name: data, Length: 394, dtype: object

In [443]:
dict = {}
count = 0
for data in x_train1:
    for item in data:
        dict[item] = 0
        count +=1
for data in x_train1:
    for item in data:
        dict[item] +=1
print("Number of words after preprocessing are",count)
dict1 = pd.DataFrame(dict.items(),columns=['word','freq'])
unique_words1 = len(dict1.index)
alpha = 1
total_words1 = count
print(np.sum(dict1['freq']))
print("Total number of unique words are ",unique_words1)
temp1 = total_words1 + unique_words1
dict1['prob'] = (dict1['freq'] + alpha)/(temp1)
print(dict1)

Number of words after preprocessing are 2013
2013
Total number of unique words are  849
        word  freq      prob
0    battery    13  0.004892
1    holding     2  0.001048
2       well    28  0.010133
3      phone    64  0.022711
4     sturdy     4  0.001747
..       ...   ...       ...
844   barely     1  0.000699
845     took     1  0.000699
846  started     1  0.000699
847   making     1  0.000699
848  feature     1  0.000699

[849 rows x 3 columns]


In [444]:
dict = {}
count = 0
for data in x_train0:
    for item in data:
        dict[item] = 0
        count +=1
for data in x_train0:
    for item in data:
            dict[item] +=1
print("Number of words after preprocessing are",count)
dict0 = pd.DataFrame(dict.items(),columns=['word','freq'])
unique_words0 = len(dict0.index)
alpha = 1
total_words0 = count
# print("Total number of unique words are ",unique_words)
temp0 = total_words0 + unique_words0
dict0['prob'] = (dict0['freq'] + alpha)/(temp0)
print(dict0)

Number of words after preprocessing are 2157
                word  freq      prob
0        essentially     2  0.000944
1     communications     1  0.000629
2               tool     1  0.000629
3        communicate     1  0.000629
4               came     2  0.000944
...              ...   ...       ...
1017        recessed     1  0.000629
1018         bargain     1  0.000629
1019      microsofts     1  0.000629
1020              []     1  0.000629
1021            snap     1  0.000629

[1022 rows x 3 columns]


In [445]:
x_test = pd.DataFrame(x_test,columns=['data','prob1','prob0'])
x_test['prob1'] = 0.0
x_test['prob0'] = 0.0
x_test = x_test.reset_index(inplace=False,drop=True)
i=0
for data in x_test['data']:
    for item in data:
        if item in dict1.values:
#             print(item)
            x_test['prob1'][i] += dict1.loc[dict1['word']==item,'prob']
        else:
            x_test['prob1'][i] += 1/(temp1)
    i += 1
# print(x_test)   
i=0
for data in x_test['data']:
    for item in data:
        if item in dict0.values:
#             print(item)
            x_test['prob0'][i] += dict0.loc[dict0['word']==item,'prob']
        else:
            x_test['prob0'][i] += 1/(temp0)
    i += 1
print(x_test)   
            
            

/home/badri/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/badri/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/badri/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/badr

                                                  data     prob1     prob0
0                  [ive, nearly, years, worked, great]  0.033194  0.007235
1                              [sound, quality, great]  0.039133  0.009751
2    [speaker, low, quality, making, ring, tones, s...  0.020964  0.012583
3    [small, sleek, impressive, looking, practical,...  0.005590  0.006291
4                                      [poor, product]  0.010482  0.009751
..                                                 ...       ...       ...
195                              [dont, make, mistake]  0.002795  0.010381
196  [research, development, division, obviously, k...  0.002096  0.002831
197  [file, browser, offers, options, one, needshan...  0.031796  0.007864
198                                [would, reccommend]  0.005590  0.005033
199  [worked, well, paired, treo, w, skype, headset...  0.029001  0.015099

[200 rows x 3 columns]


In [446]:
ans = x_test['prob1']>=x_test['prob0']
ans = ans.astype(int)
ans = np.array(ans)
y_test = np.array(y_test)

In [447]:
print(ans)
print(y_test)

[1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1
 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1
 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1
 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0
 1 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 0 0 1 1 1
 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1
 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1
 0 1 0 1 0 1 1 1 0 1 0 1 1 0 1]


In [448]:
print(np.sum(ans==y_test))

142


In [449]:
print(y_test.size)

200
